# Feladatok generáslása
szöveggeneráló modellek tanítására

In [1]:
from datasets import load_from_disk

dataset = load_from_disk('datasets/base/classification_dataset')

<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow._fs.FileInfo size changed, may indicate binary incompatibility. Expected 64 from C header, got 88 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow._fs.FileSelector size changed, may indicate binary incompatibility. Expected 48 from C header, got 72 from PyObject


In [2]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'title', 'description', 'keywords', 'label', 'url', 'authors', 'date', 'kmonitor_description', 'kmonitor_title', 'kmonitor_tags', 'kmonitor_institutes', 'kmonitor_people', 'is_hand_annoted', 'kmonitor_places', 'kmonitor_collections'],
        num_rows: 43060
    })
    validation: Dataset({
        features: ['text', 'title', 'description', 'keywords', 'label', 'url', 'authors', 'date', 'kmonitor_description', 'kmonitor_title', 'kmonitor_tags', 'kmonitor_institutes', 'kmonitor_people', 'is_hand_annoted', 'kmonitor_places', 'kmonitor_collections'],
        num_rows: 6598
    })
    test: Dataset({
        features: ['text', 'title', 'description', 'keywords', 'label', 'url', 'authors', 'date', 'kmonitor_description', 'kmonitor_title', 'kmonitor_tags', 'kmonitor_institutes', 'kmonitor_people', 'is_hand_annoted', 'kmonitor_places', 'kmonitor_collections'],
        num_rows: 3618
    })
})

## Cikk klasszifikációja

In [3]:
article_classification_prompt = '''[korrupció klasszifikáció]
{title}

{description}

{text}

cimkék: {keywords}

###

téma: {theme}
'''

In [4]:
article_classification_prompt = '''[korrupció klasszifikáció]
{title}

{description}

cimkék: {keywords}

###

téma: {theme}
'''

In [5]:
def generate_article_classification_task(article):
    article['keywords'] = ', '.join(article['keywords'])
    article['theme'] = ['egyéb', 'korrupció'][article['label']]
    article['description'] = article['description'].replace(article['title'], '').replace(' - ', ' ').strip()
    article['text'] = article['text'].replace(article['description'], '').replace(' - ', ' ').strip()
    article['text'] = article['text'][:article['text'].find('\n')]
    return {'text': article_classification_prompt.format(**article), 'type': 'article_classification'}

    k = 0
print(generate_article_classification_task(dataset['train'][1])['text'])

[korrupció klasszifikáció]
Kinéztek a repülő ablakán és egy észak-koreai rakétát láttak

Hoppá, az ott nem egy ballisztikus rakéta?

cimkék: ballisztikus rakéta, észak-korea, fegyverkezés, külföld

###

téma: egyéb



In [6]:
# TODO ez szebb lenne format string-el, de akkor 20000. iteracional kicrashel valami key errorral, ami egy windowsos registry keynek tunik, WTF (linuxon vagyok)
def generate_article_classification_task_b(articles):
    results = {'text': [], 'label': [], 'type': []}
    for i in range(len(articles['text'])):
        articles['keywords'][i] = ', '.join(articles['keywords'][i])
        articles['description'][i] = articles['description'][i].replace(articles['title'][i], '').replace(' - ', ' ').strip()
        articles['text'][i] = articles['text'][i].replace(articles['description'][i], '').replace(' - ', ' ').strip()
        n = 0
        for paragraph in articles['text'][i].splitlines():
            if paragraph.strip() == articles['description'][i]:
                continue
            if n == 5:
                break
            if len(paragraph.strip()) > 250:
                n += 1
                text = '''[korrupció klasszifikáció]
'''+articles['title'][i]+'''

'''+articles['description'][i]+'''

'''+paragraph+'''

cimkék: '''+articles['keywords'][i]+'''

###

téma: '''+['egyéb', 'korrupció'][articles['label'][i]]+'''
'''
                results['text'].append(text)
                results['type'].append('article_classification')
                results['label'].append(articles['label'][i])
    return results

#print(generate_article_classification_task(dataset['train'][1])['text'])

In [7]:
article_classification_task_dataset = dataset.map(generate_article_classification_task, remove_columns=['text', 'title', 'description', 'keywords', 'url', 'authors', 'date', 'kmonitor_description', 'kmonitor_title', 'kmonitor_tags', 'kmonitor_institutes', 'kmonitor_people', 'is_hand_annoted', 'kmonitor_places', 'kmonitor_collections'])

Map:   0%|          | 0/43060 [00:00<?, ? examples/s]

Map:   0%|          | 0/6598 [00:00<?, ? examples/s]

Map:   0%|          | 0/3618 [00:00<?, ? examples/s]

In [8]:
article_classification_task_dataset = article_classification_task_dataset #.remove_columns('theme')

In [9]:
print(article_classification_task_dataset['train'][1]['text'])

[korrupció klasszifikáció]
Kinéztek a repülő ablakán és egy észak-koreai rakétát láttak

Hoppá, az ott nem egy ballisztikus rakéta?

cimkék: ballisztikus rakéta, észak-korea, fegyverkezés, külföld

###

téma: egyéb



In [10]:
article_classification_task_dataset.save_to_disk('datasets/tasks/article_classification_task_dataset')

Saving the dataset (0/1 shards):   0%|          | 0/43060 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/6598 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3618 [00:00<?, ? examples/s]

## Releváns személyek adathalmaz

In [11]:
entity_dataset = load_from_disk('datasets/intermediate/entity_dataset')

In [12]:
def make_people_dataset(articles):
    results = {'text': [], 'corrupt_people': [], 'not_corrupt_people': []}
    for i in range(len(articles['text'])):
        if articles['entities'][i] is None:
            continue
        entities = [tuple(entity) for entity in articles['entities'][i]]
        if articles['kmonitor_people'][i]:
            has_missing = False
            people = [p if ' (' not in p else p[:p.find(' (')] for p in articles['kmonitor_people'][i]]
            corrupt_people = set()
            not_corrupt_people = set()
            for person in people:
                isin = False
                original_name = ''
                for ent, label in entities:
                    if person.lower().replace('-', '').replace('dr. ', '') == ent.lower().replace('-', '').replace('dr. ', ''):
                        isin = True
                        original_name = ent
                        break
                if not isin:
                    if not has_missing:
                        print(i, articles['title'][i])
                        print(articles['url'][i])
                    print('-', person, 'not in')
                    has_missing = True
                else:
                    corrupt_people.add(original_name)
            for ent, label in entities:
                correct = True
                for person in people:
                    if ent.lower().replace('-', '').replace('dr. ', '') in person.lower().replace('-', '').replace('dr. ', '') or label != 'PER' or ' ' not in ent:
                        correct = False
                        break
                if correct:
                    not_corrupt_people.add(ent)
            if not has_missing:
                results['text'].append(articles["text"][i])
                results['corrupt_people'].append(list(corrupt_people))
                results['not_corrupt_people'].append(list(not_corrupt_people))
    return results

In [13]:
people_dataset = entity_dataset.map(make_people_dataset, batched=True, remove_columns=dataset['train'].column_names+['entities'])

In [14]:
people_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'corrupt_people', 'not_corrupt_people'],
        num_rows: 11372
    })
    validation: Dataset({
        features: ['text', 'corrupt_people', 'not_corrupt_people'],
        num_rows: 1446
    })
    test: Dataset({
        features: ['text', 'corrupt_people', 'not_corrupt_people'],
        num_rows: 748
    })
})

In [15]:
people_dataset.save_to_disk('datasets/intermediate/people_dataset')

Saving the dataset (0/1 shards):   0%|          | 0/11372 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1446 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/748 [00:00<?, ? examples/s]

## Személy-klasszifikációs feladatok generálása

In [16]:
people_dataset = load_from_disk('datasets/intermediate/people_dataset')

In [17]:
people_classification_prompt = '''[személy klasszifikáció]
{text}

###

összes: {all}
korrupcióban érintett: {corrupt}
'''

In [18]:
def generate_people_classification_task(articles):
    result = {'text': [], 'type': []}
    for i in range(len(articles['text'])):
        article = {key: articles[key][i] for key in articles}
        article['corrupt'] = ', '.join(sorted(article['corrupt_people']))
        article['other'] = ', '.join(sorted(article['not_corrupt_people']))
        article['all'] = ', '.join(sorted(article['corrupt_people'] + article['not_corrupt_people']))
        result['type'].append('people_classification')
        result['text'].append(people_classification_prompt.format(**article))
    return result

In [19]:
people_classification_task_dataset = people_dataset.map(generate_people_classification_task, batched=True, remove_columns=people_dataset['train'].column_names)

In [20]:
print(people_classification_task_dataset['train'][0]['text'])

[személy klasszifikáció]
A 8. rendű vádlott egy szerelmi kaland miatt bonyolódott a Zuschlag-ügybe, de egyébként undorodik a mutyizástól és állítja, hogy meditációs tréningen volt, amikor egy pályázatot valaki aláírt a nevében. A per kezd tragikomédiába fordulni.
A 12. és a 8. rendű vádlott meghallgatásával folytatódott pénteken a Zuschlag-per Kecskeméten. Vámosi Zsolt 8. rendű vádlott írásban tett vallomást a bíróság előtt. A harmincnégy éves férfi – aki a grafikai bérmunkákkal foglakozó 3D Brigád Zrt. elnöke, valamint a VMI Capital Befektetési Kft. ügyvezetője – visszautasította a csalás vádját és felmentését kérte bíróságtól.
Vallomása szerint a rendszerváltás első éveiben, 17 évesen indította az Ifjú Zöldek Mozgalmát, majd részt vett az Ifjú Szocialisták néhány rendezvényén, s ezek egyikén az első szerelmével ismerkedett meg. Később vállalkozni kezdett és elfordult a politikától.
A rengeteg munka miatt azonban a magánélete tönkrement, nem maradt ideje a társasági életre, ezért 2000

In [21]:
people_classification_task_dataset.save_to_disk('datasets/tasks/people_classification_task_dataset')

Saving the dataset (0/1 shards):   0%|          | 0/11372 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1446 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/748 [00:00<?, ? examples/s]

## Intézmény klasszifikációja

In [22]:
from datasets import load_from_disk
entity_dataset = load_from_disk('datasets/intermediate/entity_dataset')

In [23]:
set([tuple(entity) for entities in entity_dataset['train']['entities'][:10] for entity in entities if entity[1] == 'ORG'])

{('3D Brigád Zrt.', 'ORG'),
 ('APEH', 'ORG'),
 ('Alkotmánybíróság', 'ORG'),
 ('Aquifex', 'ORG'),
 ('B Build & Trade Kft.', 'ORG'),
 ('BKV', 'ORG'),
 ('BKV Zrt.', 'ORG'),
 ('Balatoni Turizmus Szövetség', 'ORG'),
 ('Budapest Bank', 'ORG'),
 ('Continental', 'ORG'),
 ('Everling Építő Termelő és Szolgáltató Kft.', 'ORG'),
 ('F.F.V.D. Bt.', 'ORG'),
 ('FEH', 'ORG'),
 ('Facebook', 'ORG'),
 ('Felügyeleti és Ellenőrzési Hivatal', 'ORG'),
 ('Főpolgármesteri Hivatal', 'ORG'),
 ('Fővárosi Főügyészség', 'ORG'),
 ('Google', 'ORG'),
 ('Grabarics Kft.', 'ORG'),
 ('Géplak Kft.', 'ORG'),
 ('Ifjú Szocialisták', 'ORG'),
 ('Ifjú Zöldek Mozgalma', 'ORG'),
 ('Index', 'ORG'),
 ('KACU Kft.', 'ORG'),
 ('Kantar', 'ORG'),
 ('Kantar Media', 'ORG'),
 ('Kapolyi-féle Kárpát Energo', 'ORG'),
 ('Kárpát Energo', 'ORG'),
 ('Kárpát Energó', 'ORG'),
 ('Lakosok Egyesülete', 'ORG'),
 ('Levegő Munkacsoport', 'ORG'),
 ('MH', 'ORG'),
 ('MNB', 'ORG'),
 ('MVM', 'ORG'),
 ('Magyar Fejlesztési Bank', 'ORG'),
 ('Magyar Hírlap', 'ORG')

In [24]:
def make_institutions_dataset(articles):
    results = {'text': [], 'corrupt_institutions': [], 'not_corrupt_institutions': []}
    for i in range(len(articles['text'])):
        if articles['entities'][i] is None:
            continue
        entities = [tuple(entity) for entity in articles['entities'][i]]
        if articles['kmonitor_institutes'][i]:
            has_missing = False
            institutions = [p if ' (' not in p else p[:p.find(' (')] for p in articles['kmonitor_institutes'][i]]
            corrupt_institutions = set()
            not_corrupt_institutions = set()
            for institution in institutions:
                isin = False
                original_name = ''
                for ent, label in entities:
                    if institution.lower().replace(' zrt.', '').replace(' kft.', '') == ent.lower().replace(' zrt.', '').replace(' kft.', ''):
                        isin = True
                        original_name = ent
                        break
                if not isin:
                    if not has_missing:
                        print(i, articles['title'][i])
                        print(articles['url'][i])
                    print('-', institution, 'not in')
                    has_missing = True
                else:
                    corrupt_institutions.add(original_name)
            for ent, label in entities:
                correct = True
                for institution in institutions:
                    if ent.lower().replace(' zrt.', '').replace(' kft.', '') in institution.lower().replace(' zrt.', '').replace(' kft.', '') or label != 'ORG' or ' ' not in ent:
                        correct = False
                        break
                if correct:
                    not_corrupt_institutions.add(ent)
            if not has_missing:
                results['text'].append(articles["text"][i])
                results['corrupt_institutions'].append(list(corrupt_institutions))
                results['not_corrupt_institutions'].append(list(not_corrupt_institutions))
    return results

In [25]:
institutions_dataset = entity_dataset.map(make_institutions_dataset, batched=True, remove_columns=dataset['train'].column_names+['entities'])

In [26]:
institutions_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'corrupt_institutions', 'not_corrupt_institutions'],
        num_rows: 7189
    })
    validation: Dataset({
        features: ['text', 'corrupt_institutions', 'not_corrupt_institutions'],
        num_rows: 920
    })
    test: Dataset({
        features: ['text', 'corrupt_institutions', 'not_corrupt_institutions'],
        num_rows: 492
    })
})

In [27]:
institutions_dataset.save_to_disk('datasets/intermediate/institutions_dataset')

Saving the dataset (0/1 shards):   0%|          | 0/7189 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/920 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/492 [00:00<?, ? examples/s]

## Intézményklasszifikációs feladatok generálása

In [28]:
institutions_dataset = load_from_disk('datasets/intermediate/institutions_dataset')

In [29]:
institution_classification_prompt = '''[intézmény klasszifikáció]
{text}

###

összes: {all}
korrupcióban érintett: {corrupt}
'''

In [30]:
def generate_institution_classification_task(articles):
    result = {'text': [], 'type': []}
    for i in range(len(articles['text'])):
        article = {key: articles[key][i] for key in articles}
        article['corrupt'] = ', '.join(sorted(article['corrupt_institutions']))
        article['other'] = ', '.join(sorted(article['not_corrupt_institutions']))
        article['all'] = ', '.join(sorted(article['corrupt_institutions'] + article['not_corrupt_institutions']))
        result['type'].append('institution_classification')
        result['text'].append(people_classification_prompt.format(**article))
    return result

In [31]:
institution_classification_task_dataset = institutions_dataset.map(generate_institution_classification_task, batched=True, remove_columns=institutions_dataset['train'].column_names)

In [32]:
institution_classification_task_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'type'],
        num_rows: 7189
    })
    validation: Dataset({
        features: ['text', 'type'],
        num_rows: 920
    })
    test: Dataset({
        features: ['text', 'type'],
        num_rows: 492
    })
})

In [33]:
institution_classification_task_dataset.save_to_disk('datasets/tasks/institution_classification_task_dataset')

Saving the dataset (0/1 shards):   0%|          | 0/7189 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/920 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/492 [00:00<?, ? examples/s]

## Helyszínek klasszifikációja

In [34]:
set([tuple(entity) for entities in entity_dataset['train']['entities'][:10] for entity in entities if entity[1] == 'LOC'])

{('Balatonfüred', 'LOC'),
 ('Balkán', 'LOC'),
 ('Bosznia', 'LOC'),
 ('Dárdai Pál Labdarúgó Akadémia', 'LOC'),
 ('Horvátország', 'LOC'),
 ('In-shore', 'LOC'),
 ('Kecskemét', 'LOC'),
 ('Nagymező utca', 'LOC'),
 ('Paks', 'LOC'),
 ('Pogány', 'LOC'),
 ('Pécs', 'LOC'),
 ('Szerbia', 'LOC'),
 ('Verseny u.', 'LOC')}

In [35]:
set([entity for entities in entity_dataset['train']['kmonitor_places'][:10] for entity in entities])

{'Baranya megye', 'Bp. VI. kerület', 'Békés megye', 'Pécs', 'belföld'}

## Címkék generálása

In [36]:
keyword_dataset = dataset.filter(lambda article: article['label'] == 1)

In [37]:
keyword_generation_prompt = '''[címkék generálása]
{title}

{description}

{text}

cimkék: {keywords}

###

korrupciós címkék: {kmonitor_tags}
'''

In [38]:
def generate_keyword_generation_task(article):
    article['keywords'] = ', '.join(article['keywords'])
    article['kmonitor_tags'] = ', '.join(article['kmonitor_tags'])
    return {'text': keyword_generation_prompt.format(**article), 'type': 'keyword_generation'}

print(generate_keyword_generation_task(keyword_dataset['train'][1])['text'])

[címkék generálása]
A belső elhárítás tudott a békési rendőrügyekről

A Rendvédelmi Szervek Védelmi Szolgálata (RSZVSZ), több, hivatali úton érkezett jelzés nyomán vizsgálta a békési rendőrvezetők és az alvilág kapcsolatát - értesült a Magyar

A Rendvédelmi Szervek Védelmi Szolgálata (RSZVSZ), több, hivatali úton érkezett jelzés nyomán vizsgálta a békési rendőrvezetők és az alvilág kapcsolatát - értesült a Magyar Hírlap.
Tonhauser László közölte, a békési ügyekről neki is voltak információi, amelyekről tájékoztatta a belső elhárítást. Az ORFK szóvivője bejelentette: várhatóan ezen a héten lezárul a '95-ös olajügyekkel kapcsolatos nyomozások és a Békés megyei rendőrségnél történt rendkívüli halálesetek ügyében indított vizsgálat. Bócz Endre, fővárosi főügyész megkapta Pallag László kisgazda-képviselő feljelentését, de mint mondta: nincs az ügyben illetékessége, mert ha valaki a helyi szerveket elfogultnak tartja, a legfőbb ügyészhez kellett volna fordulnia.
Orbán Viktor a tegnapi kormán

In [39]:
keyword_generation_task_dataset = keyword_dataset.map(generate_keyword_generation_task, remove_columns=keyword_dataset['train'].column_names)

In [40]:
keyword_generation_task_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'type'],
        num_rows: 26661
    })
    validation: Dataset({
        features: ['text', 'type'],
        num_rows: 3410
    })
    test: Dataset({
        features: ['text', 'type'],
        num_rows: 1720
    })
})

In [41]:
keyword_generation_task_dataset.save_to_disk('datasets/tasks/keyword_generation_task_dataset')

Saving the dataset (0/1 shards):   0%|          | 0/26661 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3410 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1720 [00:00<?, ? examples/s]

## helyreigazítás detektálása

## ügyek detektálása

## kapcsolatok ent

## Összes feladat

In [42]:
article_classification_task_dataset = load_from_disk('datasets/tasks/article_classification_task_dataset/')
institution_classification_task_dataset = load_from_disk('datasets/tasks/institution_classification_task_dataset//')
keyword_generation_task_dataset = load_from_disk('datasets/tasks/keyword_generation_task_dataset/')
people_classification_task_dataset = load_from_disk('datasets/tasks/people_classification_task_dataset//')

In [43]:
article_classification_task_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'theme', 'type'],
        num_rows: 43060
    })
    validation: Dataset({
        features: ['text', 'label', 'theme', 'type'],
        num_rows: 6598
    })
    test: Dataset({
        features: ['text', 'label', 'theme', 'type'],
        num_rows: 3618
    })
})